In [1]:
import os

In [2]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Mlops\\Malicious_QR_Code_Detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Mlops\\Malicious_QR_Code_Detection'

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data: str
    test_data: str
    model_name: str
   

In [5]:
from src.MaliciousQRCodeDetection.constants import *
from src.MaliciousQRCodeDetection.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__ (
            self,
            config_filepath = CONFIG_FILE_PATH,
            ):
        
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_training

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data=config.train_data,
            test_data=config.test_data,
            model_name=config.model_name,)
        
        return model_trainer_config


In [11]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2
import pandas as pd

In [14]:
from src.MaliciousQRCodeDetection.logging.logger import logger
from src.MaliciousQRCodeDetection.exception import MaliciousQRException

In [24]:
class ModelTrainer:
    def __init__ (self, config: ModelTrainerConfig):
        self.config = config

    def Train(self):

        train_data = pd.read_csv(self.config.train_data)
        logger.info('Train data read Successfully')

        X_train = train_data.drop(columns=['result'])
        y_train = train_data['result']

        model = Sequential()

        model.add(Dense(32, activation='relu', input_shape=(23,), kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.2))  
        
        model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))        
        model.add(Dropout(0.2))  

        model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.01)))   
        model.add(Dropout(0.2))  

        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer= 'adam',loss='binary_crossentropy',metrics=['acc'])

        checkpointer = ModelCheckpoint(
            self.config.model_name,    # Filepath to save the model
            monitor='val_acc',    # Metric to monitor for improvement
            mode='max',           # Save model when the monitored metric is maximized
            verbose=2,            # Print messages when saving the model
            save_best_only=True,   # Only save the model if it's the best so far
        )


        history=model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2 , callbacks=[checkpointer])
        logger.info('Model Training Completed ......')

In [16]:
import sys

In [25]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.Train()
except Exception as e:
    logger.info('Problem during model training...')
    raise MaliciousQRException(e,sys)

[ 2024-12-29 14:19:04,961] 17 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-12-29 14:19:04,962] 31 root - INFO - Created directory at: artifacts
[ 2024-12-29 14:19:04,963] 31 root - INFO - Created directory at: artifacts/model_training
[ 2024-12-29 14:19:06,494] 8 root - INFO - Train data read Successfully
Epoch 1/10


c:\Users\fifty\anaconda3\envs\malicious_url_detection\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3119/3163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.9481 - loss: 0.3181
Epoch 1: val_acc improved from -inf to 0.99564, saving model to artifacts/model_training/model.keras
3163/3163 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - acc: 0.9486 - loss: 0.3158 - val_acc: 0.9956 - val_loss: 0.0673
Epoch 2/10
3128/3163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.9899 - loss: 0.0820
Epoch 2: val_acc did not improve from 0.99564
3163/3163 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - acc: 0.9899 - loss: 0.0820 - val_acc: 0.9953 - val_loss: 0.0581
Epoch 3/10
3130/3163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.9905 - loss: 0.0709
Epoch 3: val_acc did not improve from 0.99564
3163/3163 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - acc: 0.9905 - loss: 0.0709 - val_acc: 0.9956 - val_loss: 0.0506
Epoch 4/10
3137/3163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.9902 - loss: 0.0655
Epoch 4: val_acc improved from 0.99564 to 0.99581, saving model to artifacts/model_training/model.keras
3163/3163 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - acc: 0.990